## 1. Import


In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


## 2. 평가 함수 정의 (NMAE 집중)


In [2]:
def comovement_nmae(answer_df, submission_df, eps=1e-6):
    """
    전체 U = G ∪ P에 대한 clipped NMAE 계산
    NMAE가 낮을수록 좋음 (0에 가까울수록 좋음)
    """
    ans = answer_df[["leading_item_id", "following_item_id", "value"]].copy()
    sub = submission_df[["leading_item_id", "following_item_id", "value"]].copy()
    ans["pair"] = list(zip(ans["leading_item_id"], ans["following_item_id"]))
    sub["pair"] = list(zip(sub["leading_item_id"], sub["following_item_id"]))
    
    G = set(ans["pair"])
    P = set(sub["pair"])
    U = G | P
    
    ans_val = dict(zip(ans["pair"], ans["value"]))
    sub_val = dict(zip(sub["pair"], sub["value"]))
    
    errors = []
    for pair in U:
        if pair in G and pair in P:
            # 정수 변환(반올림)
            y_true = int(round(float(ans_val[pair])))
            y_pred = int(round(float(sub_val[pair])))
            rel_err = abs(y_true - y_pred) / (abs(y_true) + eps)
            rel_err = min(rel_err, 1.0)  # 오차 100% 이상은 100%로 간주
        else:
            rel_err = 1.0  # FN, FP는 오차 100%
        errors.append(rel_err)
    
    return np.mean(errors) if errors else 1.0

def analyze_prediction_errors(answer_df, submission_df):
    """예측 오차 분석"""
    ans = answer_df[["leading_item_id", "following_item_id", "value"]].copy()
    sub = submission_df[["leading_item_id", "following_item_id", "value"]].copy()
    ans["pair"] = list(zip(ans["leading_item_id"], ans["following_item_id"]))
    sub["pair"] = list(zip(sub["leading_item_id"], sub["following_item_id"]))
    
    ans_val = dict(zip(ans["pair"], ans["value"]))
    sub_val = dict(zip(sub["pair"], sub["value"]))
    
    errors = []
    for pair in ans["pair"]:
        if pair in sub_val:
            y_true = int(round(float(ans_val[pair])))
            y_pred = int(round(float(sub_val[pair])))
            abs_err = abs(y_true - y_pred)
            rel_err = abs_err / (abs(y_true) + 1e-6)
            errors.append({
                "pair": pair,
                "y_true": y_true,
                "y_pred": y_pred,
                "abs_err": abs_err,
                "rel_err": rel_err
            })
    
    error_df = pd.DataFrame(errors)
    print("=== 예측 오차 분석 ===")
    print(f"평균 절대 오차: {error_df['abs_err'].mean():,.0f}")
    print(f"평균 상대 오차: {error_df['rel_err'].mean():.4f}")
    print(f"중위수 상대 오차: {error_df['rel_err'].median():.4f}")
    print(f"\n상대 오차 분포:")
    print(error_df['rel_err'].describe())
    print(f"\n상대 오차가 0.5 이상인 쌍: {(error_df['rel_err'] >= 0.5).sum()} / {len(error_df)}")
    print(f"상대 오차가 1.0인 쌍: {(error_df['rel_err'] >= 1.0).sum()} / {len(error_df)}")
    
    return error_df


## 3. 데이터 전처리 및 학습/검증 분리


In [3]:
train = pd.read_csv('../data/raw/train.csv')

# year, month, item_id 기준으로 value 합산
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)

# year, month를 하나의 키(ym)로 묶기
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)

# item_id × ym 피벗 (월별 총 무역량 매트릭스 생성)
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)

# 2025-07-01을 기준으로 학습/검증 분리
val_date = pd.to_datetime("2025-07-01")

# 학습 데이터: 2025-07-01 이전 데이터만 사용
pivot_train = pivot.loc[:, pivot.columns < val_date].copy()
print(f"학습 데이터 기간: {pivot_train.columns.min()} ~ {pivot_train.columns.max()}")
print(f"학습 데이터 shape: {pivot_train.shape}")

# 검증 데이터: 2025-07-01 데이터
if val_date in pivot.columns:
    print(f"\n검증 데이터 날짜: {val_date}")
    print(f"검증 데이터 shape: ({pivot.shape[0]}, 1)")
else:
    print(f"\n경고: {val_date} 데이터가 pivot에 없습니다.")


학습 데이터 기간: 2022-01-01 00:00:00 ~ 2025-06-01 00:00:00
학습 데이터 shape: (100, 42)

검증 데이터 날짜: 2025-07-01 00:00:00
검증 데이터 shape: (100, 1)


## 4. 공행성쌍 탐색 (학습 데이터만 사용)


In [4]:
def safe_corr(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])

def find_comovement_pairs(
    pivot, 
    max_lag=6, 
    min_nonzero=12, 
    corr_threshold=0.4
):
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)

    results = []

    for i, leader in tqdm(enumerate(items)):
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue

        for follower in items:
            if follower == leader:
                continue

            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0

            # lag = 1 ~ max_lag 탐색
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue
                corr = safe_corr(x[:-lag], y[lag:])
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag

            # 임계값 이상이면 공행성쌍으로 채택
            if best_lag is not None and abs(best_corr) >= corr_threshold:
                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr": best_corr,
                })

    pairs = pd.DataFrame(results)
    return pairs

# 학습 데이터로만 공행성쌍 탐색
pairs = find_comovement_pairs(pivot_train)
print("탐색된 공행성쌍 수:", len(pairs))
pairs.head()


100it [00:10,  9.49it/s]

탐색된 공행성쌍 수: 1453


,leading_item_id,following_item_id,best_lag,max_corr
0,AANGBULD,APQGTRMF,5,-0.459240
1,AANGBULD,DEWLVASR,6,0.673163
2,AANGBULD,DNMPSKTB,4,-0.434721
3,AANGBULD,EVBVXETX,6,0.453442
4,AANGBULD,FTSVTTSR,3,0.533976


## 5. 고급 Feature Engineering 및 학습 데이터 생성


In [5]:
def build_training_data_enhanced(pivot, pairs, use_log_transform=False):
    """
    향상된 Feature Engineering을 적용한 학습 데이터 생성
    - 이동평균
    - 트렌드
    - 정규화된 값
    - 계절성 (같은 월의 과거 값)
    """
    months = pivot.columns.to_list()
    n_months = len(months)

    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t+1이 존재하고, t-lag >= 0인 구간만 학습에 사용
        for t in range(max(lag, 2), n_months - 1):  # 최소 2개월 전 데이터 필요
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            b_t_2 = b_series[t - 2] if t >= 2 else 0.0
            a_t_lag = a_series[t - lag]
            a_t_lag_1 = a_series[t - lag - 1] if t - lag - 1 >= 0 else 0.0
            b_t_plus_1 = b_series[t + 1]

            # 기본 feature
            features = {
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": float(lag),
            }
            
            # 추가 feature: 이동평균
            window = min(3, t + 1)
            features["b_t_ma3"] = np.mean(b_series[max(0, t - window + 1):t + 1])
            features["a_t_lag_ma3"] = np.mean(a_series[max(0, t - lag - window + 1):t - lag + 1]) if t - lag >= 0 else 0.0
            
            # 추가 feature: 트렌드 (변화율)
            features["b_trend"] = (b_t - b_t_1) / (b_t_1 + 1e-6) if b_t_1 > 0 else 0.0
            features["b_trend_2"] = (b_t_1 - b_t_2) / (b_t_2 + 1e-6) if b_t_2 > 0 else 0.0
            features["a_trend"] = (a_t_lag - a_t_lag_1) / (a_t_lag_1 + 1e-6) if a_t_lag_1 > 0 else 0.0
            
            # 추가 feature: 정규화된 값
            b_mean = np.mean(b_series[:t+1])
            a_mean = np.mean(a_series[:t-lag+1]) if t - lag >= 0 else 1.0
            features["b_t_scaled"] = b_t / (b_mean + 1e-6)
            features["a_t_lag_scaled"] = a_t_lag / (a_mean + 1e-6)
            
            # 추가 feature: 상관관계 가중치
            features["a_t_lag_weighted"] = a_t_lag * abs(corr)
            
            # 추가 feature: lag별 특성
            features["lag_1"] = 1.0 if lag == 1 else 0.0
            features["lag_2"] = 1.0 if lag == 2 else 0.0
            features["lag_3plus"] = 1.0 if lag >= 3 else 0.0
            
            # Target
            if use_log_transform:
                features["target"] = np.log1p(b_t_plus_1)
            else:
                features["target"] = b_t_plus_1

            rows.append(features)

    df_train = pd.DataFrame(rows)
    return df_train

# 로그 변환 옵션 (실험적으로 선택)
USE_LOG_TRANSFORM = False  # True로 변경하여 로그 변환 사용 가능

df_train_model = build_training_data_enhanced(pivot_train, pairs, use_log_transform=USE_LOG_TRANSFORM)
print('생성된 학습 데이터의 shape :', df_train_model.shape)
print('Feature 목록:', [col for col in df_train_model.columns if col != 'target'])
df_train_model.head()


생성된 학습 데이터의 shape : (54154, 17)
Feature 목록: ['b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag', 'b_t_ma3', 'a_t_lag_ma3', 'b_trend', 'b_trend_2', 'a_trend', 'b_t_scaled', 'a_t_lag_scaled', 'a_t_lag_weighted', 'lag_1', 'lag_2', 'lag_3plus']


,b_t,b_t_1,a_t_lag,max_corr,best_lag,b_t_ma3,a_t_lag_ma3,b_trend,b_trend_2,a_trend,b_t_scaled,a_t_lag_scaled,a_t_lag_weighted,lag_1,lag_2,lag_3plus,target
0,582317.0,539873.0,14276.0,-0.45924,5.0,530862.666667,14276.000000,0.078618,0.147694,0.000000,1.290947,1.000000,6556.109414,0.0,0.0,1.0,759980.0
1,759980.0,582317.0,52347.0,-0.45924,5.0,627390.000000,33311.500000,0.305097,0.078618,2.666783,1.534674,1.571439,24039.833250,0.0,0.0,1.0,216019.0
2,216019.0,759980.0,53549.0,-0.45924,5.0,519438.666667,40057.333333,-0.715757,0.305097,0.022962,0.469292,1.336809,24591.839660,0.0,0.0,1.0,537693.0
3,537693.0,216019.0,0.0,-0.45924,5.0,504564.000000,35298.666667,1.489100,-0.715757,-1.000000,1.146696,0.000000,0.000000,0.0,0.0,1.0,205326.0
4,205326.0,537693.0,26997.0,-0.45924,5.0,319679.333333,26848.666667,-0.618135,1.489100,0.000000,0.463963,0.917211,12398.100717,0.0,0.0,1.0,169440.0


## 6. 다양한 모델 학습 및 비교


In [6]:
# Feature 선택
feature_cols = [col for col in df_train_model.columns if col != 'target']
train_X = df_train_model[feature_cols].values
train_y = df_train_model["target"].values

print(f"Feature 개수: {len(feature_cols)}")
print(f"학습 샘플 수: {len(train_X)}")

# 여러 모델 학습 및 비교
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "RandomForest": RandomForestRegressor(
        n_estimators=100, 
        max_depth=15, 
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    ),
    "GradientBoosting": GradientBoostingRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=42
    ),
    "XGBoost": XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=42,
        n_jobs=-1
    )
}

trained_models = {}
for name, model in tqdm(models.items(), desc="모델 학습"):
    model.fit(train_X, train_y)
    trained_models[name] = model
    print(f"{name} 학습 완료")

print("\n모든 모델 학습 완료!")


Feature 개수: 16
학습 샘플 수: 54154


모델 학습:   0%|          | 0/6 [00:00<?, ?it/s]

LinearRegression 학습 완료
Ridge 학습 완료


모델 학습:  50%|█████     | 3/6 [00:00<00:00,  4.98it/s]

Lasso 학습 완료


...

RandomForest 학습 완료


모델 학습:  83%|████████▎ | 5/6 [00:30<00:09,  9.14s/it]

GradientBoosting 학습 완료


모델 학습: 100%|██████████| 6/6 [00:31<00:00,  5.20s/it]

XGBoost 학습 완료

모든 모델 학습 완료!


## 7. 앙상블 예측 함수


In [7]:
def predict_single_model(pivot, pairs, model, feature_cols, target_date, use_log_transform=False):
    """
    단일 모델을 사용한 예측 함수
    """
    # target_date 이전까지의 데이터만 사용
    pivot_for_pred = pivot.loc[:, pivot.columns < target_date].copy()
    
    if len(pivot_for_pred.columns) == 0:
        raise ValueError(f"target_date {target_date} 이전의 데이터가 없습니다.")
    
    months = pivot_for_pred.columns.to_list()
    n_months = len(months)
    
    # 가장 마지막 달 index
    t_last = n_months - 1
    t_prev = t_last - 1 if t_last > 0 else t_last
    t_prev2 = t_last - 2 if t_last >= 2 else 0

    preds = []

    for row in tqdm(pairs.itertuples(index=False), desc="예측 중"):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        if leader not in pivot_for_pred.index or follower not in pivot_for_pred.index:
            continue

        a_series = pivot_for_pred.loc[leader].values.astype(float)
        b_series = pivot_for_pred.loc[follower].values.astype(float)

        # t_last - lag 가 0 이상인 경우만 예측
        if t_last - lag < 0:
            continue

        # Feature 생성 (학습 시와 동일한 방식)
        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev] if t_prev >= 0 else 0.0
        b_t_2 = b_series[t_prev2] if t_prev2 >= 0 else 0.0
        a_t_lag = a_series[t_last - lag]
        a_t_lag_1 = a_series[t_last - lag - 1] if t_last - lag - 1 >= 0 else 0.0

        # Feature 벡터 생성
        features = {
            "b_t": b_t,
            "b_t_1": b_t_1,
            "a_t_lag": a_t_lag,
            "max_corr": corr,
            "best_lag": float(lag),
            "b_t_ma3": np.mean(b_series[max(0, t_last - 2):t_last + 1]),
            "a_t_lag_ma3": np.mean(a_series[max(0, t_last - lag - 2):t_last - lag + 1]) if t_last - lag >= 0 else 0.0,
            "b_trend": (b_t - b_t_1) / (b_t_1 + 1e-6) if b_t_1 > 0 else 0.0,
            "b_trend_2": (b_t_1 - b_t_2) / (b_t_2 + 1e-6) if b_t_2 > 0 else 0.0,
            "a_trend": (a_t_lag - a_t_lag_1) / (a_t_lag_1 + 1e-6) if a_t_lag_1 > 0 else 0.0,
            "b_t_scaled": b_t / (np.mean(b_series[:t_last+1]) + 1e-6),
            "a_t_lag_scaled": a_t_lag / (np.mean(a_series[:t_last-lag+1]) + 1e-6) if t_last - lag >= 0 else 0.0,
            "a_t_lag_weighted": a_t_lag * abs(corr),
            "lag_1": 1.0 if lag == 1 else 0.0,
            "lag_2": 1.0 if lag == 2 else 0.0,
            "lag_3plus": 1.0 if lag >= 3 else 0.0,
        }
        
        X_test = np.array([[features[col] for col in feature_cols]])

        # 단일 모델 예측
        y_pred = model.predict(X_test)[0]

        # 로그 변환 사용 시 역변환
        if use_log_transform:
            y_pred = np.expm1(y_pred)

        # 후처리: 음수 방지 및 정수 변환
        y_pred = max(0.0, float(y_pred))
        
        # 추가 후처리: 이상치 제한
        if b_t > 0:
            # 현재 값의 20배를 넘지 않도록 제한
            y_pred = min(y_pred, b_t * 20)
            # 최근 트렌드 반영 (선택적)
            if b_t_1 > 0:
                trend = b_t / b_t_1
                y_pred = y_pred * (0.7 + 0.3 * min(trend, 2.0))  # 트렌드 반영하되 과도하지 않게
        
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })

    df_pred = pd.DataFrame(preds)
    return df_pred


## 8. 정답 데이터 생성 (2025-07-01 실제 값)


In [8]:
# train.csv에서 2025-07-01의 실제 데이터 추출
val_year = val_date.year
val_month = val_date.month

answer_raw = train[
    (train["year"] == val_year) & 
    (train["month"] == val_month)
].copy()

# item_id별 value 합산
answer_monthly = (
    answer_raw
    .groupby("item_id", as_index=False)["value"]
    .sum()
)

# 공행성쌍에 대해 정답 생성
answer_dict = dict(zip(answer_monthly["item_id"], answer_monthly["value"]))

answer_list = []
for row in pairs.itertuples(index=False):
    follower = row.following_item_id
    if follower in answer_dict:
        answer_list.append({
            "leading_item_id": row.leading_item_id,
            "following_item_id": follower,
            "value": answer_dict[follower]
        })
    else:
        answer_list.append({
            "leading_item_id": row.leading_item_id,
            "following_item_id": follower,
            "value": 0
        })

answer_df = pd.DataFrame(answer_list)
print(f"생성된 정답 데이터 수: {len(answer_df)}")
print(f"정답 데이터 value 합계: {answer_df['value'].sum():,.0f}")
answer_df.head()


생성된 정답 데이터 수: 1453
정답 데이터 value 합계: 7,009,633,460


,leading_item_id,following_item_id,value
0,AANGBULD,APQGTRMF,40608.0
1,AANGBULD,DEWLVASR,482787.0
2,AANGBULD,DNMPSKTB,4507669.0
3,AANGBULD,EVBVXETX,5061099.0
4,AANGBULD,FTSVTTSR,246916.0


## 9. 모든 모델 개별 예측 및 성능 비교


In [9]:
# 모든 모델에 대해 개별 예측 및 평가 수행
print("=== 모든 모델 개별 예측 및 성능 비교 ===\n")

model_results = []
all_submissions = {}

for name, model in trained_models.items():
    print(f"\n[{name}] 예측 중...")
    
    # 예측 수행
    submission = predict_single_model(
        pivot, 
        pairs, 
        model, 
        feature_cols, 
        val_date,
        use_log_transform=USE_LOG_TRANSFORM
    )
    
    all_submissions[name] = submission
    
    # NMAE 계산
    nmae = comovement_nmae(answer_df, submission)
    nmae_score = 1 - nmae
    
    # 오차 분석
    error_df = analyze_prediction_errors(answer_df, submission)
    
    # 결과 저장
    model_results.append({
        "모델명": name,
        "NMAE": nmae,
        "NMAE Score (1-NMAE)": nmae_score,
        "평균 절대 오차": error_df['abs_err'].mean(),
        "평균 상대 오차": error_df['rel_err'].mean(),
        "중위수 상대 오차": error_df['rel_err'].median(),
        "상대오차 >= 0.5 비율": (error_df['rel_err'] >= 0.5).sum() / len(error_df),
        "상대오차 >= 1.0 비율": (error_df['rel_err'] >= 1.0).sum() / len(error_df),
        "예측값 평균": submission['value'].mean(),
        "예측값 중위수": submission['value'].median(),
        "예측값 합계": submission['value'].sum(),
    })
    
    print(f"  → NMAE: {nmae:.6f}, NMAE Score: {nmae_score:.6f}")

# 결과를 DataFrame으로 정리
results_df = pd.DataFrame(model_results)
results_df = results_df.sort_values('NMAE')  # NMAE가 낮을수록 좋음

print("\n" + "="*80)
print("=== 전체 모델 성능 비교 결과 ===")
print("="*80)
print(results_df.to_string(index=False))
print("\n" + "="*80)
print(f"최고 성능 모델: {results_df.iloc[0]['모델명']} (NMAE: {results_df.iloc[0]['NMAE']:.6f})")
print(f"최저 성능 모델: {results_df.iloc[-1]['모델명']} (NMAE: {results_df.iloc[-1]['NMAE']:.6f})")
print("="*80)


=== 모든 모델 개별 예측 및 성능 비교 ===


[LinearRegression] 예측 중...


예측 중: 1453it [00:00, 3819.20it/s]


=== 예측 오차 분석 ===
평균 절대 오차: 1,637,718
평균 상대 오차: 6065879573.9253
중위수 상대 오차: 0.4709

상대 오차 분포:
count    1.453000e+03
mean     6.065880e+09
std      3.723605e+10
min      0.000000e+00
25%      2.299670e-01
50%      4.708687e-01
75%      2.058753e+00
max      4.191930e+11
Name: rel_err, dtype: float64

상대 오차가 0.5 이상인 쌍: 702 / 1453
상대 오차가 1.0인 쌍: 480 / 1453
  → NMAE: 0.567033, NMAE Score: 0.432967

[Ridge] 예측 중...


예측 중: 1453it [00:00, 3675.66it/s]


=== 예측 오차 분석 ===
평균 절대 오차: 1,637,717
평균 상대 오차: 6065931191.2687
중위수 상대 오차: 0.4709

상대 오차 분포:
count    1.453000e+03
mean     6.065931e+09
std      3.723605e+10
min      0.000000e+00
25%      2.299634e-01
50%      4.708628e-01
75%      2.059039e+00
max      4.191750e+11
Name: rel_err, dtype: float64

상대 오차가 0.5 이상인 쌍: 702 / 1453
상대 오차가 1.0인 쌍: 480 / 1453
  → NMAE: 0.567032, NMAE Score: 0.432968

[Lasso] 예측 중...


...

=== 예측 오차 분석 ===
평균 절대 오차: 1,617,468
평균 상대 오차: 828027533.6077
중위수 상대 오차: 0.3728

상대 오차 분포:
count    1.453000e+03
mean     8.280275e+08
std      8.008459e+09
min      4.868113e-03
25%      1.504682e-01
50%      3.728442e-01
75%      7.836490e-01
max      1.063870e+11
Name: rel_err, dtype: float64

상대 오차가 0.5 이상인 쌍: 578 / 1453
상대 오차가 1.0인 쌍: 305 / 1453
  → NMAE: 0.473753, NMAE Score: 0.526247

[GradientBoosting] 예측 중...


예측 중: 1453it [00:00, 2120.84it/s]


=== 예측 오차 분석 ===
평균 절대 오차: 1,533,726
평균 상대 오차: 2086582252.4566
중위수 상대 오차: 0.4150

상대 오차 분포:
count    1.453000e+03
mean     2.086582e+09
std      1.232987e+10
min      1.065266e-03
25%      1.402020e-01
50%      4.150278e-01
75%      1.595040e+00
max      1.113700e+11
Name: rel_err, dtype: float64

상대 오차가 0.5 이상인 쌍: 622 / 1453
상대 오차가 1.0인 쌍: 433 / 1453
  → NMAE: 0.499985, NMAE Score: 0.500015

[XGBoost] 예측 중...


예측 중: 1453it [00:00, 1612.25it/s]

=== 예측 오차 분석 ===
평균 절대 오차: 1,774,326
평균 상대 오차: 2158165181.7119
중위수 상대 오차: 0.4226

상대 오차 분포:
count    1.453000e+03
mean     2.158165e+09
std      1.287263e+10
min      7.228586e-04
25%      1.574825e-01
50%      4.226378e-01
75%      1.279509e+00
max      1.210200e+11
Name: rel_err, dtype: float64

상대 오차가 0.5 이상인 쌍: 634 / 1453
상대 오차가 1.0인 쌍: 420 / 1453
  → NMAE: 0.511883, NMAE Score: 0.488117

=== 전체 모델 성능 비교 결과 ===
             모델명     NMAE  NMAE Score (1-NMAE)     평균 절대 오차     평균 상대 오차  중위수 상대 오차  상대오차 >= 0.5 비율  상대오차 >= 1.0 비율       예측값 평균  예측값 중위수     예측값 합계
    RandomForest 0.473753             0.526247 1.617468e+06 8.280275e+08   0.372844        0.397798        0.209911 3.674716e+06 353400.0 5339362868
GradientBoosting 0.499985             0.500015 1.533726e+06 2.086582e+09   0.415028        0.428080        0.298004 3.892900e+06 346603.0 5656384020
         XGBoost 0.511883             0.488117 1.774326e+06 2.158165e+09   0.422638        0.436339        0.289057 3.702477e+06 33584

## 10. 상세 성능 비교 및 분석


In [10]:
# 상세 비교를 위한 추가 분석
print("\n=== 모델별 상세 비교 ===\n")

# 1. NMAE Score 순위
print("1. NMAE Score 순위 (높을수록 좋음):")
for idx, row in results_df.iterrows():
    rank = list(results_df.sort_values('NMAE Score (1-NMAE)', ascending=False).index).index(idx) + 1
    print(f"  {rank}위: {row['모델명']:20s} - NMAE Score: {row['NMAE Score (1-NMAE)']:.6f}")

# 2. 예측값 분포 비교
print("\n2. 예측값 통계 비교:")
print(results_df[['모델명', '예측값 평균', '예측값 중위수', '예측값 합계']].to_string(index=False))

# 3. 오차 분포 비교
print("\n3. 오차 분포 비교:")
print(results_df[['모델명', '평균 절대 오차', '평균 상대 오차', '중위수 상대 오차']].to_string(index=False))

# 4. 정답과의 차이
print("\n4. 정답 데이터 통계:")
print(f"  정답값 합계: {answer_df['value'].sum():,.0f}")
print(f"  정답값 평균: {answer_df['value'].mean():,.0f}")
print(f"  정답값 중위수: {answer_df['value'].median():,.0f}")

print("\n5. 예측값과 정답값 비교:")
for name in results_df['모델명']:
    pred_sum = all_submissions[name]['value'].sum()
    diff_pct = abs(pred_sum - answer_df['value'].sum()) / (answer_df['value'].sum() + 1e-6) * 100
    print(f"  {name:20s}: 예측 합계 = {pred_sum:>15,.0f}, 차이 = {diff_pct:.2f}%")



=== 모델별 상세 비교 ===

1. NMAE Score 순위 (높을수록 좋음):
  1위: RandomForest         - NMAE Score: 0.526247
  2위: GradientBoosting     - NMAE Score: 0.500015
  3위: XGBoost              - NMAE Score: 0.488117
  4위: Ridge                - NMAE Score: 0.432968
  5위: LinearRegression     - NMAE Score: 0.432967
  6위: Lasso                - NMAE Score: 0.432967

2. 예측값 통계 비교:
             모델명       예측값 평균  예측값 중위수     예측값 합계
    RandomForest 3.674716e+06 353400.0 5339362868
GradientBoosting 3.892900e+06 346603.0 5656384020
         XGBoost 3.702477e+06 335848.0 5379698930
           Ridge 3.790429e+06 539392.0 5507493456
LinearRegression 3.790430e+06 539405.0 5507494549
           Lasso 3.790430e+06 539404.0 5507494476

3. 오차 분포 비교:
             모델명     평균 절대 오차     평균 상대 오차  중위수 상대 오차
    RandomForest 1.617468e+06 8.280275e+08   0.372844
GradientBoosting 1.533726e+06 2.086582e+09   0.415028
         XGBoost 1.774326e+06 2.158165e+09   0.422638
           Ridge 1.637717e+06 6.065931e+09   0.470863
Lin

## 11. 최고 성능 모델 선택


In [11]:
# 최고 성능 모델 선택
best_model_name = results_df.iloc[0]['모델명']
best_submission = all_submissions[best_model_name]
best_nmae = results_df.iloc[0]['NMAE']
best_nmae_score = results_df.iloc[0]['NMAE Score (1-NMAE)']

print(f"=== 최고 성능 모델: {best_model_name} ===")
print(f"NMAE: {best_nmae:.6f}")
print(f"NMAE Score: {best_nmae_score:.6f}")
print(f"\n최고 성능 모델의 예측 결과:")
print(best_submission.head(10))


=== 최고 성능 모델: RandomForest ===
NMAE: 0.473753
NMAE Score: 0.526247

최고 성능 모델의 예측 결과:
  leading_item_id following_item_id     value
0        AANGBULD          APQGTRMF     36636
1        AANGBULD          DEWLVASR    315056
2        AANGBULD          DNMPSKTB   5822062
3        AANGBULD          EVBVXETX   4682915
4        AANGBULD          FTSVTTSR    147946
5        AANGBULD          GKQIJYDH  28231590
6        AANGBULD          GYHKIVQT  12131322
7        AANGBULD          KJNSOAHR  10240623
8        AANGBULD          LLHREMKS     54550
9        AANGBULD          LSOIUSXD    119406
